In [244]:
import tensorflow

In [245]:
import keras

In [246]:

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


In [247]:
#import tensorflow as tf
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15191968168891436147
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11185114317
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15686890113988362573
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [248]:
#tbCallBack = keras.callbacks.TensorBoard(log_dir='/data/tb-logs', histogram_freq=0, write_graph=True, write_images=True)


In [249]:
img_width, img_height = 128, 128
train_data_dir = "/data/train_quad"
validation_data_dir = "/data/train_quad"
nb_train_samples = 4573
nb_validation_samples = 506 

In [250]:
nr_categories = 4
batch_size = 32
epochs = 50

In [251]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
#from keras.models import load_model
#model = load_model("/data/vgg16_1.h5")

In [252]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [253]:
# # Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
# for layer in model.layers[:12]:
#     print(layer.name)
#     layer.trainable = False

In [254]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
#x = Dense(1024, activation="relu")(x)
predictions = Dense(nr_categories, activation="softmax")(x)

In [255]:
model_final = Model(input = model.input, output = predictions)
#model_final = model

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [256]:
# Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
for layer in model_final.layers[:12]:
    print(layer.name)
    layer.trainable = False

input_9
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool


In [257]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [259]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=["accuracy"])


In [260]:
train__val_datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.1)

In [261]:
#validation_datagen = ImageDataGenerator(rescale = 1./255)

In [262]:
img_width, img_height = 128, 128
train_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

Found 4573 images belonging to 4 classes.


In [263]:
validation_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        subset="validation",
        batch_size=batch_size,
        class_mode='categorical')

Found 506 images belonging to 4 classes.


In [264]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_quad_cat_1_200_300_400_p2_sgd_mom0.9.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')


In [265]:
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

In [242]:
#batch 16

In [22]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
286/285 [==============================] - 123s 429ms/step - loss: 0.5622 - acc: 0.7754 - val_loss: 0.3410 - val_acc: 0.8874

Epoch 00001: val_acc improved from -inf to 0.88735, saving model to vgg16_quad_cat_1_200_300_400_p2.h5
Epoch 2/50
286/285 [==============================] - 117s 408ms/step - loss: 0.1855 - acc: 0.9323 - val_loss: 0.2357 - val_acc: 0.9328

Epoch 00002: val_acc improved from 0.88735 to 0.93281, saving model to vgg16_quad_cat_1_200_300_400_p2.h5
Epoch 3/50
286/285 [==============================] - 116s 406ms/step - loss: 0.1194 - acc: 0.9580 - val_loss: 0.3024 - val_acc: 0.9150

Epoch 00003: val_acc did not improve from 0.93281
Epoch 4/50
286/285 [==============================] - 116s 406ms/step - loss: 0.0892 - acc: 0.9685 - val_loss: 0.2336 - val_acc: 0.9229

Epoch 00004: val_acc did not improve from 0.93281
Epoch 5/50
 20/285 [=>............................] - ETA: 1:00 - loss: 0.0784 - acc: 0.9812

KeyboardInterrupt: 

In [ ]:
#batch 256

In [26]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
18/17 [==============================] - 135s 8s/step - loss: 0.0529 - acc: 0.9844 - val_loss: 0.1957 - val_acc: 0.9447

Epoch 00001: val_acc improved from 0.93281 to 0.94466, saving model to vgg16_quad_cat_1_200_300_400_p2.h5
Epoch 2/50
 6/17 [=========>....................] - ETA: 43s - loss: 0.0413 - acc: 0.9876

KeyboardInterrupt: 

In [ ]:
#adam

In [115]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
143/142 [==============================] - 120s 838ms/step - loss: 1.2476 - acc: 0.5654 - val_loss: 0.8304 - val_acc: 0.6364

Epoch 00001: val_acc improved from -inf to 0.63636, saving model to vgg16_quad_cat_1_200_300_400_p2_adam.h5
Epoch 2/50
143/142 [==============================] - 113s 790ms/step - loss: 0.7199 - acc: 0.6761 - val_loss: 0.7691 - val_acc: 0.6917

Epoch 00002: val_acc improved from 0.63636 to 0.69170, saving model to vgg16_quad_cat_1_200_300_400_p2_adam.h5
Epoch 3/50
 63/142 [============>.................] - ETA: 47s - loss: 0.6805 - acc: 0.6920

KeyboardInterrupt: 

In [ ]:
#rmsprop

In [90]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
143/142 [==============================] - 121s 848ms/step - loss: 10.1410 - acc: 0.3669 - val_loss: 10.1933 - val_acc: 0.3676

Epoch 00001: val_acc improved from -inf to 0.36759, saving model to vgg16_quad_cat_1_200_300_400_p2_rmsp.h5
Epoch 2/50
 21/142 [===>..........................] - ETA: 46s - loss: 10.8893 - acc: 0.3244

KeyboardInterrupt: 

In [ ]:
# adagrad

In [162]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
143/142 [==============================] - 120s 836ms/step - loss: 10.2328 - acc: 0.3600 - val_loss: 10.1933 - val_acc: 0.3676

Epoch 00001: val_acc improved from -inf to 0.36759, saving model to vgg16_quad_cat_1_200_300_400_p2_adagrad.h5
Epoch 2/50
 29/142 [=====>........................] - ETA: 50s - loss: 10.3343 - acc: 0.3588

KeyboardInterrupt: 

In [ ]:
#Adadelta

In [139]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
143/142 [==============================] - 121s 845ms/step - loss: 11.6146 - acc: 0.2744 - val_loss: 11.6585 - val_acc: 0.2767

Epoch 00001: val_acc improved from -inf to 0.27668, saving model to vgg16_quad_cat_1_200_300_400_p2_adelta.h5
Epoch 2/50
 55/142 [==========>...................] - ETA: 50s - loss: 11.4567 - acc: 0.2892

KeyboardInterrupt: 

In [ ]:
# sgd

In [186]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
143/142 [==============================] - 119s 829ms/step - loss: 0.8366 - acc: 0.6690 - val_loss: 0.4811 - val_acc: 0.8340

Epoch 00001: val_acc improved from -inf to 0.83399, saving model to vgg16_quad_cat_1_200_300_400_p2_sgd.h5
Epoch 2/50
 44/142 [========>.....................] - ETA: 54s - loss: 0.3359 - acc: 0.8686

KeyboardInterrupt: 

In [ ]:
# sgd 0.001

In [209]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
143/142 [==============================] - 119s 830ms/step - loss: 0.4274 - acc: 0.8349 - val_loss: 0.4304 - val_acc: 0.8597

Epoch 00001: val_acc improved from -inf to 0.85968, saving model to vgg16_quad_cat_1_200_300_400_p2_sgd_lr10th.h5
Epoch 2/50
 57/142 [==========>...................] - ETA: 49s - loss: 0.1274 - acc: 0.9587

KeyboardInterrupt: 

In [ ]:
# sgd momentum 0.5

In [ ]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

In [ ]:
# sgd 0.01

In [217]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
143/142 [==============================] - 118s 827ms/step - loss: 12.4551 - acc: 0.2176 - val_loss: 12.8053 - val_acc: 0.2055

Epoch 00001: val_acc improved from -inf to 0.20553, saving model to vgg16_quad_cat_1_200_300_400_p2_sgd_lr100th.h5
Epoch 2/50
 14/142 [=>............................] - ETA: 29s - loss: 12.9520 - acc: 0.1964

KeyboardInterrupt: 

In [ ]:
# momentum=0.5

In [225]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
143/142 [==============================] - 119s 831ms/step - loss: 12.7955 - acc: 0.2061 - val_loss: 12.8053 - val_acc: 0.2055

Epoch 00001: val_acc improved from -inf to 0.20553, saving model to vgg16_quad_cat_1_200_300_400_p2_sgd_mom0.5.h5
Epoch 2/50
 16/142 [==>...........................] - ETA: 36s - loss: 13.3163 - acc: 0.1738

KeyboardInterrupt: 

In [ ]:
# momentum=0.7

In [233]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
137/142 [===========================>..] - ETA: 4s - loss: 12.7982 - acc: 0.2060

KeyboardInterrupt: 

In [ ]:
# momentum=0.9

In [ ]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
142/142 [============================>.] - ETA: 0s - loss: 0.8722 - acc: 0.6527

In [ ]:
!ls

In [ ]:
batch_size = 16

generator = train__val_datagen.flow_from_directory(
        train_data_dir,
        target_size=(128, 128),
        batch_size=int(batch_size/10),
        class_mode=None) #,  # this means our generator will only yield batches of data, no labels
#) #shuffle=False)  # our data will be in order, so all first 1000 images will be cats, then 1000 dogs
# the predict_generator method returns the output of a model, given
# a generator that yields batches of numpy data
bottleneck_features_train = model.predict_generator(generator, 60000)
# save the output as a Numpy array
np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)

generator = train__val_datagen.flow_from_directory(
        train_data_dir,
        target_size=(128, 128),
        batch_size=int(batch_size/20),
        class_mode=None)
#) #shuffle=False)
bottleneck_features_validation = model.predict_generator(generator, 8000)
np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)

In [ ]:
train_data = np.load(open('bottleneck_features_train.npy'))
# the features were saved in order, so recreating the labels is easy
train_labels = np.array([0] * 1000 + [1] * 1000)

validation_data = np.load(open('bottleneck_features_validation.npy'))
validation_labels = np.array([0] * 400 + [1] * 400)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nr_categories, activation='softmax')) 
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=50,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights('bottleneck_fc_model.h5')

In [ ]:
print(1)